In [6]:
import cv2
import numpy as np

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('nod-tilt-detection.avi',fourcc, 20.0, (640,480))

#dinstance function
def distance(x,y):
    import math
    return math.sqrt((x[0]-y[0])**2+(x[1]-y[1])**2) 
    
#capturing the video from webcam
cap = cv2.VideoCapture(0)

#detecting corners
feature_params = dict(maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')

#getting coordinates
def get_coords(p1):
    try: return int(p1[0][0][0]), int(p1[0][0][1])
    except: return int(p1[0][0]), int(p1[0][1])

font = cv2.FONT_HERSHEY_SIMPLEX

#movement thresholds
max_head_movement = 20
movement_threshold = 50
gesture_threshold = 175

#finding the face in the image
face_found = False
frame_num = 0

x_g = 0
y_g = 0
w_g = 0
h_g = 0

while frame_num < 30:
    # Take first frame and find corners in it
    frame_num += 1
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(frame_gray, 1.3, 5)
#     print(faces)
#     print(faces[0, 0])
    for (x,y,w,h) in faces:
        x_g = faces[0, 0]
        y_g = faces[0, 1]
        w_g = faces[0, 2]
        h_g = faces[0, 3]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        face_found = True
    cv2.imshow('image',frame)
    out.write(frame)
    cv2.waitKey(1)
face_center = x_g+w_g/2, y_g+h_g/3
eyecenter = x + w / 2 
p0 = np.array([[face_center]], np.float32)

gesture = False
x_movement = 0
y_movement = 0
gesture_show = 60

while True:    
    ret,frame = cap.read()
    old_gray = frame_gray.copy()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    cv2.circle(frame, get_coords(p1), 4, (0,0,255), -1)
    cv2.circle(frame, get_coords(p0), 4, (255,0,0))
    a,b = get_coords(p0), get_coords(p1)
    x_movement += abs(a[0]-b[0])
    y_movement += abs(a[1]-b[1])

    if x_movement > gesture_threshold:
        gesture = 'Tilt'
    if y_movement > gesture_threshold:
        gesture = 'Nod'
    if gesture and gesture_show > 0:
        cv2.putText(frame,'Detected Gesture: ' + gesture,(50,50), font, 1.2,(0,0,255),3)
        gesture_show -=1
    if gesture_show == 0:
        gesture = False
        x_movement = 0
        y_movement = 0
        gesture_show = 60 #number of frames a gesture is shown
    p0 = p1
    
    cv2.imshow('image',frame)
    out.write(frame)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
        
cv2.destroyAllWindows()
cap.release()